In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import scipy
from scipy import sparse

In [ ]:
df = pd.read_csv('../data/raw/ml-10M100K/ratings.dat', 
                sep='::',  usecols=[0,1,2,3], 
                names=['user', 'movie', 'rating', 'timestamp'])

df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.sort_values(by='timestamp', inplace=True)
df.drop(['timestamp'], axis=1, inplace=True)
df = df.pivot(index='user', columns='movie', values='rating').fillna(0)
data = sparse.coo_matrix(df)


In [19]:
del df

In [20]:
data_arr = data.toarray()
tr, ts = train_test_split(data_arr, test_size=0.5, shuffle=False)
trs = sparse.coo_matrix(tr)
tss = sparse.coo_matrix(ts)
print(trs.shape)
print(tss.shape)

(34939, 10677)
(34939, 10677)


In [22]:
repr(trs)

"<34939x10677 sparse matrix of type '<class 'numpy.float64'>'\n\twith 5007872 stored elements in COOrdinate format>"

In [23]:
repr(tss)

"<34939x10677 sparse matrix of type '<class 'numpy.float64'>'\n\twith 4992182 stored elements in COOrdinate format>"

In [24]:
sparse.save_npz("../data/interim/train.npz", trs)
sparse.save_npz("../data/interim/test.npz", tss)

In [5]:
train_data = sparse.load_npz('../data/processed/train.npz')
test_data = sparse.load_npz('../data/processed/test.npz')

In [26]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, recall_at_k

# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
 # data = fetch_movielens(min_rating=1.0)
# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(trs, epochs=20, num_threads=4)

# Evaluate the trained model
test_precision = precision_at_k(model, tss, k=5).mean()

In [28]:
test_precision

0.3216692